In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from complexLayers import ComplexBatchNorm2d, ComplexConv2d, ComplexLinear
from complexLayers import ComplexDropout2d, NaiveComplexBatchNorm2d
from complexFunctions import complex_relu, complex_max_pool2d

In [2]:
batch_size = 128
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = datasets.MNIST('../data', train=True, transform=trans, download=True)
test_set = datasets.MNIST('../data', train=False, transform=trans, download=True)

train_loader = torch.utils.data.DataLoader(train_set, batch_size= batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size= batch_size, shuffle=True)

In [6]:
class ComplexNet(nn.Module):
    
    def __init__(self):
        super(ComplexNet, self).__init__()
        self.conv1 = ComplexConv2d(1, 10, 5, 1)
        self.bn  = ComplexBatchNorm2d(10)
        self.conv2 = ComplexConv2d(10, 20, 5, 1)
        self.fc1 = ComplexLinear(4*4*20, 500)
        self.dropout = ComplexDropout2d(p = 0.3)
        self.fc2 = ComplexLinear(500, 10)
             
    def forward(self,x):
        x = self.conv1(x)
        x = complex_relu(x)
        x = complex_max_pool2d(x, 2, 2)
        x = self.bn(x)
        x = self.conv2(x)
        x = complex_relu(x)
        x = complex_max_pool2d(x, 2, 2)
        x = x.view(-1,4*4*20)
        x = self.fc1(x)
        x = self.dropout(x)
        x = complex_relu(x)
        x = self.fc2(x)
        x = x.abs()
        x =  F.log_softmax(x, dim=1)
        return x
    
device = device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ComplexNet().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=5e-3, momentum=0.9)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target =data.to(device).type(torch.complex64), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {:3} [{:6}/{:6} ({:3.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(data), 
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.item())
            )

In [7]:
# Run training on 4 epochs
for epoch in range(4):
    train(model, device, train_loader, optimizer, epoch)

Train Epoch:   0 [     0/ 60000 (  0%)]	Loss: 2.788445
Train Epoch:   0 [ 12800/ 60000 ( 21%)]	Loss: 0.092633
Train Epoch:   0 [ 25600/ 60000 ( 43%)]	Loss: 0.063293
Train Epoch:   0 [ 38400/ 60000 ( 64%)]	Loss: 0.149574
Train Epoch:   0 [ 51200/ 60000 ( 85%)]	Loss: 0.106631
Train Epoch:   1 [     0/ 60000 (  0%)]	Loss: 0.069093
Train Epoch:   1 [ 12800/ 60000 ( 21%)]	Loss: 0.062164
Train Epoch:   1 [ 25600/ 60000 ( 43%)]	Loss: 0.016100
Train Epoch:   1 [ 38400/ 60000 ( 64%)]	Loss: 0.073013
Train Epoch:   1 [ 51200/ 60000 ( 85%)]	Loss: 0.029039
Train Epoch:   2 [     0/ 60000 (  0%)]	Loss: 0.021323
Train Epoch:   2 [ 12800/ 60000 ( 21%)]	Loss: 0.060060
Train Epoch:   2 [ 25600/ 60000 ( 43%)]	Loss: 0.080281
Train Epoch:   2 [ 38400/ 60000 ( 64%)]	Loss: 0.076188
Train Epoch:   2 [ 51200/ 60000 ( 85%)]	Loss: 0.049544
Train Epoch:   3 [     0/ 60000 (  0%)]	Loss: 0.017912
Train Epoch:   3 [ 12800/ 60000 ( 21%)]	Loss: 0.027665
Train Epoch:   3 [ 25600/ 60000 ( 43%)]	Loss: 0.028703
Train Epoc